In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import argparse
import joblib
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [2]:

features_adults = ['HEI_TS', 'CSES_TS', 'SCl-90 DEP', 'SCl-90 GSI', 'SCl-90 PST', 'SCl-90 TS', 'SCl-90 ANX', 'SCl-90 PSY', 'SCl-90 NST', 'SCl-90 ADD', 'SCl-90 PSDI', 'SCl-90 PAR', 'EMBU-M OI', 'SCl-90 SOM', 'EMBU-F OP', 'EMBU-F EW', 'EMBU-M EW', 'SSRS_TS', 'DES-Ⅱ_AMN', 'DES-Ⅱ_TS', 'SCl-90 OC', 'SSRS_SS', 'SCl-90 IS', 'DES-Ⅱ_ABS', 'EMBU-F OI', 'SCl-90 HOS', 'DES-Ⅱ_DPDR', 'SCQ_FAN', 'SSRS_OS', 'EMBU-F REJ', 'SCQ_RAT', 'EMBU-F PUN', 'SCQ_HS', 'SSRS_SU', 'SCQ_REP', 'SCl-90 PHOB', 'SCQ_PS', 'EMBU-M REJ', 'EMBU-F FS', 'EMBU-M PUN',  'SCQ_SB', 'EMBU-M FS']

features_teens = ['CSES_TS', 'SCl-90 DEP', 'HEI_TS', 'SCl-90 ANX', 'A-DES-Ⅱ_TS', 'A-DES-Ⅱ_PI', 'SCl-90 GSI', 'SCl-90 NST', 'SCl-90 PSY', 'EMBU-F EW', 'A-SSRS_SS', 'SCl-90 ADD', 'A-DES-Ⅱ_DPDR', 'SCl-90 PSDI', 'A-SSRS_TS', 'A-SSRS_SU', 'A-DES-Ⅱ_DA', 'SCl-90 HOS', 'EMBU-F OI', 'SCl-90 SOM', 'EMBU-M EW', 'EMBU-F PUN', 'SCl-90 TS', 'SCl-90 PHOB', 'EMBU-F OP', 'EMBU-M OI', 'SCl-90 PST', 'SCQ_FAN', 'A-SSRS_OS', 'EMBU-M PUN', 'SCQ_REP', 'SCl-90 IS', 'SCQ_PS', 'SCl-90 PAR', 'SCl-90 OC', 'SCQ_HS', 'A-DES-Ⅱ_AII', 'SCQ_SB', 'SCQ_RAT', 'EMBU-M REJ', 'EMBU-F REJ', 'EMBU-M FS', 'EMBU-F FS']

features_children = ['CSES_TS', 'HEI_TS', 'A-DES-Ⅱ_TS', 'A-SSRS_TS', 'A-DES-Ⅱ_PI', 'A-SSRS_SU', 'A-SSRS_OS', 'A-DES-Ⅱ_DA', 'A-DES-Ⅱ_AII', 'A-DES-Ⅱ_DPDR', 'SCQ_PS', 'EMBU-M PUN', 'A-SSRS_SS', 'EMBU-M OI', 'EMBU-F EW', 'EMBU-M EW', 'EMBU-M REJ', 'SCQ_SB', 'SCQ_HS', 'EMBU-F OP', 'SCQ_REP', 'EMBU-F FS', 'EMBU-F REJ', 'SCQ_RAT', 'EMBU-F PUN', 'EMBU-F OI', 'SCQ_FAN', 'EMBU-M FS']

top10_features_adults = ['HEI_TS', 'SCL-90 DEP', 'CSES_TS', 'SCL-90 PSY', 'SCL-90 ANX', 'EMBU-F EW', 'DES-Ⅱ_TS', 'DES-Ⅱ_ABS', 'DES-Ⅱ_AMN', 'EMBU-M FS']

top10_features_teens = ['SCL-90 DEP', 'A-DES-Ⅱ_PI', 'SCL-90 ANX', 'CSES_TS', 'EMBU-F EW', 'HEI_TS', 'SCL-90 PSY', 'A-DES-Ⅱ_DPDR', 'A-DES-Ⅱ_AII', 'EMBU-M FS']

top10_features_children = ['CSES_TS', 'HEI_TS', 'A-DES-Ⅱ_PI', 'A-DES-Ⅱ_AII', 'A-DES-Ⅱ_DPDR', 'EMBU-F EW', 'EMBU-F PUN', 'EMBU-M FS', 'CSQ_REP', 'A-SSRS_OS']

In [ ]:
### macro parameters
group_name = 'adults'
model_name = 'LogisticRegression'
top10 = True
top10_str = 'top10' if top10 else ''
clf_path = 'C:/Users/SCoulY/Desktop/psycology/ckpt_w_scaler/children_correct/adults/clean_adults_LogisticRegression_acc_0.91_run_123_top10.pkl'
scaler_path = 'C:/Users/SCoulY/Desktop/psycology/ckpt_w_scaler/children_correct/adults/clean_adults_scaler_top10.pkl'


exclude_cols = ['Gender']
if group_name == 'adults':
    if top10:
        features = top10_features_adults
    else:
        features = features_adults
elif top10 and group_name == 'teens':
    if top10:
        features = top10_features_teens
    else:
        features = features_teens
elif top10 and group_name == 'children':
    if top10:
        features = top10_features_children
    else:
        features = features_children
else:
    raise ValueError("Invalid group name or top10 setting")
scale_cols = [f for f in features if f not in exclude_cols]

if scaler_path and os.path.isfile(scaler_path):
    # Load existing scaler bundle
    scaler_bundle = joblib.load(scaler_path)
    scaler = scaler_bundle['scaler']
    saved_scale_cols = scaler_bundle.get('scale_cols', scale_cols)

else:
    scaler = StandardScaler()
    scaler.fit(X[scale_cols])


In [ ]:
# Interactive single-sample inference with ipywidgets (compact layout)
# Assumes variables defined earlier: column_names, scaler (or fallback), and a trained model path.

import ipywidgets as W
from IPython.display import display, clear_output, HTML
import joblib
import numpy as np
import pandas as pd
import math

# ---- Load trained model (adjust if needed) ----
clf = joblib.load(clf_path)

# read unscaled stats from scaler
means = scaler.mean_
std = scaler.scale_
mins = means - std * 3
maxs = means + std * 3

# Attempt to access external scaler else identity
try:
    scaler  # noqa: F821
except NameError:
    class _IdentityScaler:
        def transform(self, X):
            return X
    scaler = _IdentityScaler()

# ---------- Widget Construction ----------
sliders = {}
for i, col in enumerate(features):
    lo = float(mins[i])
    hi = float(maxs[i])
    mu = float(means[i])
    step = (hi - lo) / 200.0 if hi > lo else 0.1
    sliders[col] = W.FloatSlider(
        description=col[:16],
        value=mu,
        min=lo,
        max=hi,
        step=step,
        readout=False,
        continuous_update=False,
        layout=W.Layout(width='220px')
    )

# Readout labels placed separately for cleaner width
value_labels = {c: W.Label(f"{sliders[c].value:.2f}") for c in features}

for c, s in sliders.items():
    def _make_observer(col=c, slider=s):
        def _obs(change):
            value_labels[col].value = f"{change['new']:.2f}"
        slider.observe(_obs, names='value')
    _make_observer()

# Arrange sliders into N columns to reduce vertical scroll
N_COLS = 3 if len(features) >= 15 else 2
per_col = math.ceil(len(features) / N_COLS)
cols = []
for i in range(N_COLS):
    chunk = features[i*per_col:(i+1)*per_col]
    vb_items = [W.HBox([sliders[c], value_labels[c]]) for c in chunk]
    cols.append(W.VBox(vb_items, layout=W.Layout(margin='0 10px 0 0')))
slider_panel = W.HBox(cols)

# Accordion collapsible by default (collapsed => selected_index=None)
accordion = W.Accordion(children=[slider_panel])
accordion.set_title(0, 'Feature Inputs (expand)')
accordion.selected_index = None

# Buttons + options
predict_button = W.Button(description='Predict', button_style='primary', tooltip='Run model prediction', icon='play')
reset_button = W.Button(description='Reset Means', tooltip='Reset all sliders to mean', icon='refresh')
show_full_toggle = W.ToggleButton(value=False, description='Show Full Input', tooltip='Toggle full feature table display', icon='table')

status_dict = {0: 'Withdrawal', 1: 'Reentry'}
out = W.Output(layout={'border': '1px solid #ccc', 'padding': '6px'})

# ---------- Helper Functions ----------

def _collect_sample():
    return pd.DataFrame([{c: sliders[c].value for c in features}])


def _format_sample(sample: pd.DataFrame, show_full: bool):
    # Transpose & create tidy 2-column table
    df_t = sample.T.reset_index().rename(columns={'index': 'Feature', 0: 'Value'})
    if not show_full:
        # Show only first 15 (or all if fewer); could adapt to show changed from mean
        df_t = df_t.iloc[:15].copy()
        more = sample.shape[1] - 15
        if more > 0:
            df_t.loc[len(df_t)] = ['... (+{} more)'.format(more), '']
    style = (df_t.style.set_table_styles([
        {'selector': 'th', 'props': [('font-size', '11px'), ('text-align', 'left')]},
        {'selector': 'td', 'props': [('font-size', '11px'), ('padding', '2px 6px')]},
    ]).hide(axis='index'))
    return style


def on_predict(_):
    with out:
        clear_output(wait=True)
        sample = _collect_sample()
        sample_scaled = scaler.transform(sample[features])
        proba = clf.predict_proba(sample_scaled)[0]
        pred_idx = int(np.argmax(proba))
        pred_class = clf.classes_[pred_idx]
        display(HTML('<b>Input Sample (partial view)</b>' if not show_full_toggle.value else '<b>Input Sample (full)</b>'))
        display(_format_sample(sample, show_full_toggle.value))
        print('Prediction probabilities:')
        for cls, p in zip(clf.classes_, proba):
            print(f'  P({status_dict.get(cls, cls)}) = {p:.4f}')
        print(f'Predicted class: {status_dict.get(pred_class, pred_class)} (index {pred_idx})')


def on_reset(_):
    for c in features:
        sliders[c].value = float(means[c])
    # Trigger UI refresh if already displayed
    if out.outputs:
        on_predict(None)


def on_toggle(_):
    if out.outputs:
        on_predict(None)

predict_button.on_click(on_predict)
reset_button.on_click(on_reset)
show_full_toggle.observe(on_toggle, names='value')

# ---------- Assemble UI ----------
buttons = W.HBox([predict_button, reset_button, show_full_toggle])
ui = W.VBox([
    buttons,
    accordion,
    out
])

# Inject a little CSS to constrain overall width & font
custom_css = HTML("""
<style>
    .widget-inline-hbox .widget-label { min-width: 0 !important; }
</style>
""")

display(custom_css, ui)